In [ ]:
%%capture
!pip install -q playwright nest_asyncio
!playwright install chromium
import nest_asyncio
import asyncio
import pandas as pd
from playwright.async_api import async_playwright
import re
nest_asyncio.apply()


In [ ]:
_BASE_URL_='https://www.zenga.hu'
def url_creator(place:str="pest",prop_type:str="haz",price_min=40000000, price_max=100000000):

    return f"{_BASE_URL_}/{place}+elado+{prop_type}+ar-{price_min}-{price_max}?page="

In [ ]:
url_creator("urom")

'https://www.zenga.hu/urom+elado+haz+ar-40000000-100000000?page='

In [ ]:
async def scrape_nof_links(start_url="https://www.zenga.hu/komarom-esztergom-megye+elado+haz?page=",page_num=1):
    '''
    Az megadott strat_url oldalán lévő linkek számát adja vissza
    :param start_url: az oldal url-je
    :param page_num: az oldal száma
    '''
    start_url = start_url+str(page_num)
    links = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(start_url)
        await page.wait_for_timeout(8000)

        # Scroll, hogy több kártya betöltődjön
        await page.mouse.wheel(0, 5000)
        await page.wait_for_timeout(6000)

        div_list = await page.query_selector_all("div[data-cy='search-page-total-count']") # linkek itt vannak a html-ben
        text=await div_list[0].text_content()
        print(f"Talált linkek száma: {text}")
        number_of_links = int(re.search(r'\d+', text.replace('\u00A0', '').replace(' ', '')).group())


        await browser.close()
        return number_of_links

In [ ]:
number_of_links=await scrape_nof_links()
print(f"A keresésnek megfelelő linkek száma:{number_of_links}")

Talált linkek száma:  1 823 találat 
A keresésnek megfelelő linkek száma:1823


In [ ]:
async def scrape_zenga_links(start_url="https://www.zenga.hu/komarom-esztergom-megye+elado+haz?page=",zenga_page=1)->list:
    '''
    A zenga.hu adott keresési oldaláról (oldalszám alapján) összegyűjti az ingatlanhirdetések
    kártyalinkjeit.

    Paraméterek
    ----------
        start_url : str, alapértelmezett: "https://www.zenga.hu/komarom-esztergom-megye+elado+haz?page="
            A listaoldal alapprefixe, amely "page=" végű query-vel zárul. A függvény a megadott
            oldalszámot fűzi a végére.
        zenga_page : int, alapértelmezett: 1
            A lekérdezni kívánt oldal száma.

    Visszatérés
    -----------
        list[str]
            A hirdetéskártyák abszolút URL-jeinek listája.
    '''

    start_url = start_url+str(zenga_page)
    links = []

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto(start_url)
        await page.wait_for_timeout(8000)

        # Scroll, hogy több kártya betöltődjön
        await page.mouse.wheel(0, 5000)
        await page.wait_for_timeout(3000)

        a_links = await page.query_selector_all("a[data-cy='advert-card-link']") # linkek itt vannak a html-ben
        print(f"Talált linkek száma: {len(a_links)}")

        for a in a_links:
            href = await a.get_attribute("href")
            if href:
                links.append(_BASE_URL_ + "/" + href)

        await browser.close()
        return links

In [ ]:
def get_all_url(place:str,prop_type="haz",l_price=40000000, h_price=100000000)->list:
    '''
    A megadott hely (place), ingatlantípus (prop_type) és ársáv (HUF) alapján összegyűjti a
    zenga.hu hirdetéslistáiról az összes hirdetés URL-jét. Először lekéri a találatok
    számát, majd az összes szükséges oldalon végiglépked és kimenti a linkeket.

    Paraméterek
    -----------
    place : str
        Település/megyenév vagy slug, amit a `url_creator` elfogad (pl. "komarom-esztergom-megye").
    prop_type : str, alapértelmezett: "haz"
        Ingatlantípus (pl. "haz", "lakas", "telek").
        l_price : int, alapértelmezett: 40_000_000
        Alsó árhatár HUF-ban.
    h_price : int, alapértelmezett: 100_000_000
        Felső árhatár HUF-ban.

    Visszatérés
    -----------
    list[str]
        A talált hirdetések abszolút URL-jeinek listája.
    '''

    url=url_creator(place=place,prop_type=prop_type,price_min=l_price,price_max=h_price)
    print(url)
    zenga_links=[]

    no_links = asyncio.run(scrape_nof_links(start_url=url,page_num=1))
    # Futtatás

    for i in range(-(-no_links // 20)):
        print(f"Page: {i+1}")
        zenga_page = asyncio.run( scrape_zenga_links(start_url=url,zenga_page=i+1))
        zenga_links.extend(zenga_page)
    # Kiírás
    for i, link in enumerate(zenga_links, 1):
        print(f"{i}. {link}")
    return zenga_links


In [ ]:
zenga_links=get_all_url("torokbalint","haz",40000000,100000000) # csak teszt

https://www.zenga.hu/torokbalint+elado+haz+ar-40000000-100000000?page=
Talált linkek száma:  21 találat 
Page: 1
Talált linkek száma: 20
Page: 2
Talált linkek száma: 1
1. https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint-ofalu/8671805?page=1&pos=1&cr=79
2. https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint/8624680?page=1&pos=2&cr=11
3. https://www.zenga.hu//ingatlan/elado-ikerhaz-torokbalint/7843274?page=1&pos=3&cr=5
4. https://www.zenga.hu//ingatlan/elado-hazresz-torokbalint/8481940?page=1&pos=4
5. https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint/8626097?page=1&pos=5
6. https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint/8642644?page=1&pos=6
7. https://www.zenga.hu//ingatlan/elado-hazresz-torokbalint/8520058?page=1&pos=7
8. https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint/8649772?page=1&pos=8
9. https://www.zenga.hu//ingatlan/elado-csaladi-haz-torokbalint-annahegy/8644752?page=1&pos=9
10. https://www.zenga.hu//ingatlan/elado-csalad

In [ ]:
city="zsambek"
property="haz"

In [ ]:
zenga_links=get_all_url(city,property,20000000,100000000)
df=pd.DataFrame(zenga_links,columns=['link'])
csv_path='/content/drive/MyDrive/Zenga/'

https://www.zenga.hu/zsambek+elado+haz+ar-20000000-100000000?page=
Talált linkek száma:  37 találat 
Page: 1
Talált linkek száma: 20
Page: 2
Talált linkek száma: 17
1. https://www.zenga.hu//ingatlan/elado-csaladi-haz-zsambek/8669448?page=1&pos=1&cr=75
2. https://www.zenga.hu//ingatlan/elado-csaladi-haz-zsambek/8497907?page=1&pos=2&cr=6
3. https://www.zenga.hu//ingatlan/elado-csaladi-haz-zsambek/8659521?page=1&pos=3&cr=5
4. https://www.zenga.hu//ingatlan/elado-ikerhaz-zsambek/8564513?page=1&pos=4
5. https://www.zenga.hu//ingatlan/elado-ikerhaz-zsambek/8564520?page=1&pos=5
6. https://www.zenga.hu//ingatlan/elado-csaladi-haz-zsambek/8519461?page=1&pos=6
7. https://www.zenga.hu//ingatlan/elado-ikerhaz-zsambek/8646267?page=1&pos=7
8. https://www.zenga.hu//ingatlan/elado-ikerhaz-zsambek/8564508?page=1&pos=8
9. https://www.zenga.hu//ingatlan/elado-ikerhaz-zsambek/8564504?page=1&pos=9
10. https://www.zenga.hu//ingatlan/elado-ikerhaz-zsambek/8445100?page=1&pos=10
11. https://www.zenga.hu//ingat

In [ ]:
df.to_csv(csv_path+f'links_{city}_{property}.csv',index=False)

In [ ]:
async def scrape_ad_text(url):
    '''
    Egy zenga.hu hirdetés részletező oldaláról leolvassa a leírást és az árat és visszaadja egy listában.

    Paraméterek
    -----------
    url : str
        A hirdetés részletező oldalának abszolút URL-je.

    Visszatérés
    -----------
    list[dict[str, str]]
        Alapesetben egy elemű lista, eleme: {"url": <url>, "desc": <leírás>, "price": <ár>}.
        Hiba esetén a lista egy hibaüzenetet tartalmazó rekordot ad vissza
        (ld. Megjegyzések a kulcsnév-konzisztenciáról).
    '''

    results = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()


        try:
            print(f"🔗 {url}")
            await page.goto(url, timeout=20000)
            try:
                await page.get_by_text("Összes süti engedélyezése").click()
                print("Süti elfogadva")
            except:
                print("Süti gomb nem jelent meg")


            await page.wait_for_timeout(7000)
            #await page.screenshot(path="screenshot.png")  # DEBUG
            desc_element = await page.query_selector("span[data-cy='advert-details-description']")
            desc_text = await desc_element.inner_text() if desc_element else "Nincs szöveg"
            print(f"Szöveg: {desc_text} ")

            price_element = await page.query_selector("div[data-cy='advert-details-price']")
            price_text = await price_element.inner_text() if price_element else "Nincs ár"
            print(f"Ár: {price_text} ")


            results.append({"url": url, "desc": desc_text, "price":price_text})
        except Exception as e:
            print(f"Hiba: {str(e)}")
            results.append({"url": url, "desc": f"Hiba: {str(e)}","price":f"Hiba: {str(e)}"})

        await browser.close()

    return results  #return a list


In [ ]:
_DEBUG_=True

In [ ]:
def scrape_zenga_list(z_list):
    """
    A megadott zenga.hu hirdetés-URL-ek listáját sorban feldolgozza: mindegyikre lefuttatja a
    `scrape_ad_text` aszinkron függvényt.

    Paraméterek
    -----------
    z_list : list[str]
        Hirdetés-részletező oldalak abszolút URL-jeinek listája.

    Visszatérés
    -----------
    list[dict[str, str]]
        A hirdetésekből kinyert rekordok listája. Az egyes elemek a `scrape_ad_text` által
        visszaadott dict-ek (pl. {"url", "desc", "price"}.

    """
    out_list=[]
    for url in z_list:
        url_data = asyncio.run(scrape_ad_text(url))
        if _DEBUG_:
            print(url_data)
        out_list.extend(url_data)
    return out_list

In [ ]:
%%capture
zenga_data=scrape_zenga_list(zenga_links)

In [ ]:
df=pd.DataFrame(zenga_data)
csv_path='/content/drive/MyDrive/Zenga/'
df.to_csv(csv_path+f'zenga_data_{city}_{property}.csv',index=False)

In [ ]:
szoveg=df.desc[0]

In [ ]:
raise SystemExit("🛑 Kód megszakítva ezen a ponton.") # A következő rész a AI prompt beállításait tartalmazza

SystemExit: 🛑 Kód megszakítva ezen a ponton.

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
city="zsambek"
property="haz"

In [ ]:
#ujraépítjuk az adatokat, innen bármikor el kell indulni bármilyen csv-ből.
csv_path='/content/drive/MyDrive/Zenga/'
df_city=pd.read_csv(csv_path+f'zenga_data_{city}.csv')

In [ ]:
from openai import OpenAI
import json

In [ ]:
from google.colab import userdata
key=userdata.get('OpenAi_API')

In [ ]:
#Demo szöveg
szoveg='''
Budafok szép panorámás részén a Rózsavölgy feletti dombon, csendes mellékutcában eladó egy bájos kis ház 520 négyzetméteres telken.

A kis ház 35 négyzetméteres, egy légtérben van kialakítva benne a nappali, melyet részben nyitott válaszfallal szeparáltak el a konyha-étkező résztől. A zuhanyzós fürdő és a WC egy helyiségben található.

A víz, az áram és az internet be van kötve a házba. Gáz és csatorna az utcában. A csatorna bevezetését már kezdeményezte a szomszédos telek tulajdonosa.

A telken áll egy másik kis házikó is, ami jelenleg egy 9 négyzetméteres szobaként funkcionál.

A kert szép és gondozott, sok gyümölcsfával. Nagyobb mérete miatt sok lehetőséget rejt. Kialakítható rajta több nagy magaságyás, építhető kerti medence vagy finn szauna is.

További információkért és időpont egyeztetésért kérem hívjon telefononn.
Hitellel vásárlás esetén bankfüggetlen és DÍJMENTES hiteltanácsadó közreműködését biztosítjuk, aki a teljes folyamaton végigkíséri Önt.
Az adásvétel biztonságos és gördülékeny bonyolítása érdekében tapasztalt ingatlan szakjogászaink szolgáltatását is ajánlom.
'''

In [ ]:
szoveg=df_city.desc[0]


print(szoveg)
print(df_city.url[0])

3563 m2-es TELEK!!!!!!
Zsámbékon központi helyen, ÉPÍTÉSI ÖVEZETBEN két utcáról megközelíthető telek, rajta egy 143 m2-es felújítandó vagy bontandó házzal eladó!!! A családi ház 1960-as években épült és hat szobából két fürdőszobából egy konyhából, három kamrából áll. BEFEKTETŐKNEK ajánlom figyelmébe, ugyanis
a telek méretéből adódóan ikerház is építhető rá.
Lke-0-3 jelű építési övezetbe tartozik, 30% beépíthető!
Amennyiben felkeltette az érdeklődését és megtekintené az ingatlant hívjon bizalommal, akár hétvégén is.
https://www.zenga.hu//ingatlan/elado-csaladi-haz-zsambek/8497907?page=1&pos=1&cr=6


In [ ]:
def create_prompt(szoveg):
    prompt_message=[
            {"role": "system",
             "content": f'''
                    Te egy ingatlanhirdetés-elemző nyelvész vagy, komoly ingatlanhirdetési tapasztalattal.
                    Célod, hogy az ingatlan hirdetések tökéletesek legyenek ezért kidolgoztál egy hirdetés értékelési szempontrendszert.
                    Ezek a szempontok alapján tökéletes értékelést tudsz adni az adott hirdetés szövege alapján.
                    A szempontok:
                        1. Érthetőség
                        2. Részletesség / információtartalom
                        3. Szerkezet, logikai felépítés
                        4. Célcsoport megszólítása
                        5. Stílus és nyelvhelyesség
                        6. Előnyök kiemelése
                        7. Negatívumok őszinte kezelése
                        8. Eladásra ösztönzés'''},
            {"role": "user",
             "content": """
                            Értékeld az alábbi ingatlanhirdetés szöveget és az értékelést a scoring mezőbe helyezzed el.
                            Az alábbi szempontok alapján 1–5-ig pontozzad, és hozz létre egy összesített eredményt  1-8 közötti szempontok értékelésének átlagaként ez legyen a 9. Összesítés :
                            Majd a tudásod alapján adj egy alternatív javaslatot az eredeti szöveg javítására, hogy a lehető legjobban megfeleljen a szempontoknak.
                            Az alternatív javaslatot strukturáld, és tördeld a jobb érthetőség érdekében, de markdown formátumot ne használj!
                            Fontos, hogy a saját szempontrendszered szerint az alternatív javaslatod értékelése jobb legyen az eredeti értékelésnél!
                            A válaszod csak érvényes JSON formátumban legyen, pontosan az alábbi struktúrában:

                            {"scoring":
                                {

                                    "Érthetőség": <szám>,
                                    "Részletesség": <szám>,
                                    "Szerkezet": <szám>,
                                    "Célcsoport": <szám>,
                                    "Stílus": <szám>,
                                    "Előnyök": <szám>,
                                    "Negatívumok": <szám>,
                                    "Ösztönzés": <szám>,
                                    "Összesítés": <szám.tizedes>"
                                },
                            "proposal":"Ide kerüljön a javított szöveg javaslatod"
                            }


                    A HIRDETÉS SZÖVEGE:
                        """+f" {szoveg}"}
        ]
    return prompt_message


In [ ]:
prompt_message=create_prompt(szoveg)


In [ ]:
def get_response(szoveg:str):
    '''
    A megadott szöveghez promptot készít (`create_prompt`), elküldi az OpenAI Chat
    Completions API-nak (gpt-4o), majd a választ JSON-ként beolvassa és dict-ként visszaadja.

    Paraméterek
    -----------
    szoveg : str
        A bemeneti szöveg, amelyből a `create_prompt` összeállítja a `messages` listát.

    Visszatérés
    -----------
    dict
        A modell által visszaadott JSON objektum, Python dict-be parse-olva.
    '''
    client = OpenAI(api_key=key)
    prompt_message=create_prompt(szoveg)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=prompt_message,
        temperature=0.5,
        max_tokens=7000
    )

    content = response.choices[0].message.content

    # a gpt-4 így adja vissza a json választ, kiszedjük a jsont a felesleges keretből
    if content.startswith("```json"):
        content = content.strip("`").lstrip("json").strip()

    #
    out_dict = json.loads(content)

    return(out_dict)

In [ ]:
resp_data=get_response(prompt_message)

In [ ]:
print(resp_data)

{'scoring': {'Érthetőség': 4, 'Részletesség': 3, 'Szerkezet': 3, 'Célcsoport': 4, 'Stílus': 3, 'Előnyök': 4, 'Negatívumok': 2, 'Ösztönzés': 3, 'Összesítés': 3.3}, 'proposal': 'Eladó Zsámbékon, központi helyen található 3563 m2-es telek, amely két utcáról is megközelíthető. Az ingatlanon egy 143 m2-es, 1960-as években épült családi ház található, amely felújítandó vagy bontandó állapotú. A ház hat szobából, két fürdőszobából, egy konyhából és három kamrából áll.\n\nA telek Lke-0-3 jelű építési övezetbe tartozik, ahol 30% beépíthetőség lehetséges, így ikerház építésére is alkalmas. Kiváló lehetőség befektetők számára!\n\nHa felkeltette érdeklődését, kérem, hívjon bizalommal, akár hétvégén is, hogy megtekintse az ingatlant!'}


In [ ]:
resp_data["scoring"]

{'Érthetőség': 4,
 'Részletesség': 3,
 'Szerkezet': 3,
 'Célcsoport': 4,
 'Stílus': 3,
 'Előnyök': 4,
 'Negatívumok': 2,
 'Ösztönzés': 3,
 'Összesítés': 3.3}

In [ ]:
resp_data["proposal"]

'Eladó Zsámbékon, központi helyen található 3563 m2-es telek, amely két utcáról is megközelíthető. Az ingatlanon egy 143 m2-es, 1960-as években épült családi ház található, amely felújítandó vagy bontandó állapotú. A ház hat szobából, két fürdőszobából, egy konyhából és három kamrából áll.\n\nA telek Lke-0-3 jelű építési övezetbe tartozik, ahol 30% beépíthetőség lehetséges, így ikerház építésére is alkalmas. Kiváló lehetőség befektetők számára!\n\nHa felkeltette érdeklődését, kérem, hívjon bizalommal, akár hétvégén is, hogy megtekintse az ingatlant!'